```
This software is a part of GPU Ocean.

Copyright (C) 2019  SINTEF Digital

The aim of this notebook is to compare the efficiency of using 
different random number generators for SWE model perturbations.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Comparing Random Number Generators

The aim of this notebook is to compare the efficiency of using 
different random number generators for SWE model perturbations.

We look at initialization and simulation, and compare using curand and LCG.

## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc

import pycuda.driver as cuda
import os
import sys
import datetime
import time

from importlib import reload
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import IPythonMagic, SimReader, Observation, CDKLM16

In [ ]:
%cuda_context_handler gpu_ctx

In [ ]:
initial_memory_available, total_memory = cuda.mem_get_info()
print('Available memory: ', initial_memory_available)

# The data

The data used for this paper is available for download from [the GPU Ocean server](http://gpu-ocean.met.no:9000/gpu_ocean/DAPaper/).

Please start with downloading the files `double_jet_ensemble_init.zip` and `double_jet_truth.zip` to this folder, and unzip them so that the following lines of code passes:

In [ ]:
ensemble_init_path = os.path.abspath('double_jet_ensemble_init/')
assert len(os.listdir(ensemble_init_path)) == 102, "Ensemble init folder has wrong number of files"

truth_path = os.path.abspath('double_jet_truth/')
assert len(os.listdir(truth_path)) == 4, "Truth folder has wrong number of files"

# Read initial conditions and mean and variance 

Reading files...

In [ ]:
truth_state_filename = os.path.join(truth_path, "double_jet_case_truth.nc")
observations_filename = os.path.join(truth_path, "drifter_observations.pickle")

ensemble_init_file_name = os.path.join(ensemble_init_path, "double_jet_case_XX.nc")
num_files = 100
sim_readers = [None]*num_files
for i in range(num_files):
    sim_readers[i] = SimReader.SimNetCDFReader(ensemble_init_file_name.replace('XX', str(i).zfill(2)))

##  Testing initialization

In [ ]:
%%time
def initialize_sim_set(use_lcg=False):
    tic = time.time()
    sim_set = [None]*40
    print('started initialization with use_lcg: ', use_lcg)
    file_id_start = 0
    if use_lcg:
        file_id_start = 50
    for i in range(40):
        file_id = file_id_start + i
        sim_set[i] = CDKLM16.CDKLM16.fromfilename(gpu_ctx, 
                                                  ensemble_init_file_name.replace('XX', str(file_id).zfill(2)),
                                                  cont_write_netcdf=False,
                                                  use_lcg=use_lcg)
        if (i+1)%10 == 0:
            toc = time.time()
            print('Done with ' + str(i+1) + ' after time: ' + str(round(toc-tic,2)))
    toc = time.time()
    print('total time: ' + str(round(toc-tic,2)))
    return sim_set

In [ ]:
%%time
#curand_sims = initialize_sim_set(use_lcg=False)
curand_sims = initialize_sim_set(use_lcg=True)

curand_memory_available, total_memory = cuda.mem_get_info()
print('curand memory available: ', curand_memory_available)
print('curand memory consumption (MB): ', (initial_memory_available - curand_memory_available)/1024**2)

In [ ]:
%%time
lcg_sims = initialize_sim_set(use_lcg=True)

lcg_memory_available, total_memory = cuda.mem_get_info()
print('LCG memory available: ', lcg_memory_available)
print('LCG memory consumption (MB): ', (curand_memory_available - lcg_memory_available)/1024**2)

# Testing simulation

In [ ]:
%%time
def simulation_test(sim_set):
    tic = time.time()
    current_t = sim_set[0].t
    next_t = current_t + 2*60*60 # two hours
    print('current_t, next_t: ', (current_t, next_t))
    for i in range(len(sim_set)):
        sim_set[i].dataAssimilationStep(next_t)
        if (i+1)%10 == 0:
            toc = time.time()
            print('Done with ' + str(i+1) + ' after time: ' + str(round(toc-tic,2)))
    toc = time.time()
    print('Total simulation time with curand: ' + str(round(toc-tic,2)))

In [ ]:
%%time
simulation_test(curand_sims)

In [ ]:
%%time
simulation_test(lcg_sims)

# Sim perturbation only

In [ ]:
%%time
def perturbation_only(sim_set):
    tic = time.time()
    print('started')
    for i in range(len(sim_set)):
        for p in range(1000):
            sim_set[i].perturbState()
        if (i+1)%10 == 0:
            toc = time.time()
            print('Done with ' + str(i+1) + ' after time: ' + str(round(toc-tic,2)))
    toc = time.time()
    print('Total time: ' + str(round(toc-tic,2)))

In [ ]:
%%time
perturbation_only(curand_sims)

In [ ]:
%%time
perturbation_only(lcg_sims)